<div align="center" style="font-family:times new roman":times new roman>
<h3>Machine Learning Engineer Assessment</h3></div>
<hr>
<div align="justify" style="font-family:times new roman":times new roman>
    <h4>Full Name : Achmad Adyatma Ardi<br>
        CODE      &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;: K1/SDTECH/HR/IV/2021<br>
    </h4>
    <h5> Test Case 1 :<br>
    1. saya mau beli tiket pesawat ke surabaya untuk 2 orang.<br>
    2. beliin tiket pesawat untuk 2 orang ya, tujuannya surabaya.<br>
    3. pesankan penerbangan 2 tiket ke surabaya<br>
    <br>
    Output :<br>
    intent: beli_tiket_pesawat<br>
    destination: surabaya (place)<br>
    quantity: 2 (amount)<br>
    <br>    
    Test Case 2 :<br>
    1. beliin pulsa 10rb ke 081192327632<br>
    2. isi pulsa 10000 ke 081192327632<br>
    <br>
    Output :<br>
    intent: beli_pulsa<br>
    denom: 10000 (amount)<br>
    phone_number: 081192327632 (phone)<br>
    </h5>
</div>

<div align="center" style="font-family:times new roman"><b>1. Preparation</b>
</div>
<hr>

In [49]:
# Install spaCy
!pip install spacy -q

In [46]:
# Check the installed spaCy library
!pip show spacy -q

In [47]:
# Install pickle
!pip install pickle5 -q

In [48]:
# Check the installed pickle library
!pip show pickle5 -q

In [2]:
# Import library
import os
import spacy
from spacy import load, displacy
from spacy.tokens import DocBin
import pickle
import random
from tqdm import tqdm

<div align="center" style="font-family:times new roman"><b>2. Data train</b>
</div>
<hr>

In [ ]:
# Create annotations with the help of an external tool (open - source)

# https://tecoholic.github.io/ner-annotator/

In [3]:
# import json file
import json

os.getcwd()
f = open('annotations.json')
TRAIN_DATA = json.load(f)

In [4]:
TRAIN_DATA 

{'classes': ['PLACE', 'AMOUNT', 'PHONE'],
 'annotations': [['ibu sedang berada di Surabaya sekarang',
   {'entities': [[21, 29, 'PLACE']]}],
  ['Surabaya merupakan kota terbesar kedua setelah Jakarta',
   {'entities': [[0, 8, 'PLACE'], [47, 54, 'PLACE']]}],
  ['Surabaya adalah kota yang indah', {'entities': [[0, 8, 'PLACE']]}],
  ['Surabaya dikenal sebagai kota pahlawan', {'entities': [[0, 8, 'PLACE']]}],
  ['populasi penduduk Surabaya sekitar 3 juta orang',
   {'entities': [[18, 26, 'PLACE'], [35, 36, 'AMOUNT']]}],
  ['Terdapat 5 orang yang memiliki saham diperusahaan tersebut',
   {'entities': [[9, 10, 'AMOUNT']]}],
  ['Budi memiliki 5 buah apel', {'entities': [[14, 15, 'AMOUNT']]}],
  ['Indonesia memiliki 34 provinsi',
   {'entities': [[0, 9, 'PLACE'], [19, 21, 'AMOUNT']]}],
  ['hubungi bu Sri di nomor 081194221632', {'entities': [[24, 36, 'PHONE']]}],
  ['hubungi bu Dian di nomor 081121221639', {'entities': [[25, 37, 'PHONE']]}],
  ['hubungi pak Tobu di nomor 081421221629',
   {'en

In [10]:
# Import the DocBin object from the 'tokens' module in spacy
import spacy
from spacy.tokens import DocBin

nlp = spacy.blank("id") #load a new spacy model "Indonesia"
db = DocBin() #create a DocBin object

In [11]:
# Convert annotation file (.json) to .spacy object

for text, annot in tqdm(TRAIN_DATA['annotations']): 
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]: #add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents #label the text with the ents
    db.add(doc)

db.to_disk("./training_data.spacy") #save the DocBin object

100%|██████████████████████████████████████████| 14/14 [00:00<00:00, 319.07it/s]


In [115]:
print(db)

In [12]:
# data train configuration
!python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [13]:
#Initialize data train & pipeline
!python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy 

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2023-03-23 00:31:08,070] [INFO] Set up nlp object from config
[2023-03-23 00:31:08,082] [INFO] Pipeline: ['tok2vec', 'ner']
[2023-03-23 00:31:08,085] [INFO] Created vocabulary
[2023-03-23 00:31:08,086] [INFO] Finished initializing nlp object
[2023-03-23 00:31:08,397] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     53.21    0.00    0.00    0.00    0.00
200     200          2.64    749.37  100.00  100.00  100.00    1.00
400     400          0.00      0.00  100.00  100.00  100.00    1.00
600     600          0.00      0.00  100.00  100

<div align="center" style="font-family:times new roman"><b>3. Model validation</b>
</div>
<hr>

In [17]:
# Loading model (best model)
nlp_ner = spacy.load("/home/achmadadyatma/Documents/learncode/my-data-analyst_project/name_entity_recognition/model-best")

In [ ]:
'''
Test Case 1:
    1. saya mau beli tiket pesawat ke Surabaya untuk 2 orang
    2. beliin tiket pesawat untuk 2 orang ya, tujuannya Surabaya
    3. pesankan penerbangan 2 tiket ke Surabaya

Test Case 2:
    1. beliin pulsa 10rb ke 081192327632
    2. isi pulsa 10000 ke 081192327632
'''

In [22]:
# Test case 1
doc = nlp_ner('''
    saya mau beli tiket pesawat ke Surabaya untuk 2 orang
    beliin tiket pesawat untuk 2 orang ya, tujuannya Surabaya
    pesankan penerbangan 2 tiket ke Surabaya
''')

spacy.displacy.render(doc, style ="ent", jupyter=True)

In [29]:
# Test case 2
doc =nlp_ner('''
    beliin pulsa 10rb ke 081192327632
    isi pulsa 10000 ke 081192327632
''')

spacy.displacy.render(doc, style ="ent", jupyter=True)

<div align="center" style="font-family:times new roman"><b>4. Deploying machine learning model</b>
</div>
<hr>

In [40]:
# save machine learning model as pickle file (file.pkl)
import pickle
pickle.dump(nlp_ner,open("deploy_ml_model.pkl","wb"))

In [42]:
# load machine learning model that previously saved
model = pickle.load(open('deploy_ml_model.pkl','rb'))

In [44]:
# execute the model
doc = model(
    '''
    Ibu sedang ada di Surabaya membawa 5 anak
    hubungi bu Dian di nomor 081121221639
''')

spacy.displacy.render(doc, style ="ent", jupyter=True)

<div align="justify" style="font-family:times new roman":times new roman>
<h3>References :</h3><hr>
    https://spacy.io/usage/rule-based-matching<br>
    https://tecoholic.github.io/ner-annotator/<br>
    https://spacy.io/api/docbin<br>
    https://spacy.io/usage/models#languages<br>
    https://spacy.io/models<br>
    https://spacy.io/usage/training<br>
    https://practicaldatascience.co.uk/machine-learning/how-to-save-and-load-machine-learning-models-using-pickle<br>
</div>